# Run the MNE BIDS Pipeline

## Steps for Running this Notebook

1. Clone the Git repository of this project. (Since the Notebook is available, this step should be fulfilled.)
2. Download the data set using our script.
3. Run the patches as described in the readme file.
4. Convert the incompatible epoch label files to a BIDS compatible format using the Python script at `./src/data_handling/convert_brainvision2bids.py`
5. Run this notebook.

## Header: Load Modules, Set and Load Parameters, Define Functions
Prior to running the main steps of the pipeline, multiple steps need to be performed:

- Load modules:
  Load modules needed for all (e. g. MNE BIDS pipeline), or few (e. g. `typing`) pipeline steps. Load custom modules (e. g. `ica_helpers`)
- Set and load parameters:
  Set the relative (or absolute) path to the pipeline configuration, and load the configuration from the file.

In [1]:
# import basic EEG manipulation and analysis modules
import mne_bids_pipeline
from mne_bids import BIDSPath

# import function to load configuration from file
from mne_bids_pipeline._config_import import _import_config as getConfig

# allow for definition oRENAMEf optional parameters in a function definition
from typing import Optional

# allow for accessing the command line within Python
import sys

In [2]:
# set the file path of the main configuration file
config_path = "./mne-bids/config/mne-bids-pipeline_tests.py"

In [21]:
# for checkups, set the command for listing files directly or in a directory
ls_command = 'dir' if sys.platform.startswith('win') else 'ls'

In [4]:
# define a function which gets used in application of the ICA results to the raw data
import src.tools.ica_helpers as ih

## Pipeline

Start the pipeline running these steps:

- `preprocessing/_01_data_quality`
- `preprocessing/_04_frequency_filter`
- `preprocessing/_05_make_epochs`
- `preprocessing/_06a_run_ica`, ica labelling, `preprocessing/_07a_apply_ica`
- `preprocessing/_08_ptp_reject`
- `sensor`
- `source`

Each of the steps contains one or more checkups for validating the results of the step.

### Initial Setup

First, the selected pipeline configuration gets loaded.
Few of its entries get printed as checkup.

Then, the initial preparations for running the pipeline get started: configuration loading and directory creation.
This also is followed by a checkup.

In [3]:
# load configured settings from file
cfg = getConfig(
    config_path=config_path
)

In [15]:
# checkup: print config contents
print(
    "study name:\t\t", cfg.study_name, 
    "\nsubjects dir:\t\t", cfg.subjects_dir, 
    "\ntask:\t\t\t", cfg.task, 
    "\nconditions:\t\t", cfg.conditions, 
    "\nraw resample sfreq:\t", cfg.raw_resample_sfreq,
    "\nl/h freq:\t\t", [cfg.l_freq, cfg.h_freq],
    "\nica max iterations:\t", cfg.ica_max_iterations, 
    "\nica n components:\t", cfg.ica_n_components, 
    "\ndecoding csp freqs:\t", cfg.decoding_csp_freqs
)

study name:		 EEG-course-project 
subjects dir:		 ../data/ds003702/ 
task:			 SocialMemoryCuing 
conditions:		 ['avatar', 'sticks'] 
raw resample sfreq:	 100.0 
l/h freq:		 [1.0, 30.0] 
ica max iterations:	 3000 
ica n components:	 0.95 
decoding csp freqs:	 {'theta': [3, 7], 'alpha': [8, 12]}


In [16]:
# checkup: # print end of path to the stored bids root path
cfg.bids_root.parts[-2:]

('data', 'ds003702')

In [17]:
curr_steps = "init"
!mne_bids_pipeline --config {config_path} --steps {curr_steps}

┌────────┬ Welcome aboard MNE-BIDS-Pipeline! 👋 ────────────────────────────────
│09:10:37│ 📝 Using configuration: ./mne-bids/config/mne-bids-pipeline_tests.py
└────────┴ 
┌────────┬ init/_01_init_derivatives_dir ───────────────────────────────────────
│09:10:37│ ✅ Output directories already exist …
└────────┴ done (1s)
┌────────┬ init/_02_find_empty_room ────────────────────────────────────────────
│09:10:38│ ⏩ Skipping, empty-room data only relevant for MEG …
└────────┴ done (1s)
┌────────┬ init/_01_init_derivatives_dir ───────────────────────────────────────
│09:10:38│ ✅ Output directories already exist …
└────────┴ done (1s)
┌────────┬ init/_02_find_empty_room ────────────────────────────────────────────
│09:10:38│ ⏩ Skipping, empty-room data only relevant for MEG …
└────────┴ done (1s)


In [22]:
# checkup: check, whether output directory was created
#          The derivatives directory should exist and contain a subfolder per selected subject.
#          The subjects subfolders should be empty if no data were alread generated.
curr_bids_root = cfg.bids_root.__str__()
!"{ls_command}" "{curr_bids_root}"/derivatives/mne-bids-pipeline/

_cache			  sub-13  sub-26  sub-39
dataset_description.json  sub-14  sub-27  sub-40
sub-01			  sub-15  sub-28  sub-41
sub-02			  sub-16  sub-29  sub-43
sub-03			  sub-17  sub-30  sub-44
sub-04			  sub-18  sub-31  sub-45
sub-05			  sub-19  sub-32  sub-46
sub-06			  sub-20  sub-33  sub-48
sub-07			  sub-21  sub-34  sub-49
sub-09			  sub-22  sub-35  sub-50
sub-10			  sub-23  sub-36  task-SocialMemoryCuing_log.xlsx
sub-11			  sub-24  sub-37  task-SocialMemoryCuing_log.xlsx.lock
sub-12			  sub-25  sub-38


### Pre Processing: Data Quality
Next, an investsigation on the provided data set is started.
Per subject, a report gets created.
A simple checkup is used to print an excerpt of one of the checkups.

In [19]:
curr_steps = "preprocessing/_01_data_quality"
!mne_bids_pipeline --config {config_path} --steps {curr_steps}

┌────────┬ Welcome aboard MNE-BIDS-Pipeline! 👋 ────────────────────────────────
│09:10:47│ 📝 Using configuration: ./mne-bids/config/mne-bids-pipeline_tests.py
└────────┴ 
┌────────┬ init/_01_init_derivatives_dir ───────────────────────────────────────
│09:10:47│ ✅ Output directories already exist …
└────────┴ done (1s)
┌────────┬ init/_02_find_empty_room ────────────────────────────────────────────
│09:10:47│ ⏩ Skipping, empty-room data only relevant for MEG …
└────────┴ done (1s)
┌────────┬ preprocessing/_01_data_quality ──────────────────────────────────────
│09:10:49│ ✅ sub-04 Computation unnecessary (cached) …
│09:10:49│ ✅ sub-02 Computation unnecessary (cached) …
│09:10:49│ ✅ sub-07 Computation unnecessary (cached) …
│09:10:49│ ✅ sub-09 Computation unnecessary (cached) …
│09:10:49│ ✅ sub-10 Computation unnecessary (cached) …
│09:10:50│ ✅ sub-11 Computation unnecessary (cached) …
│09:10:50│ ✅ sub-12 Computation unnecessary (cached) …
│09:10:50│ ✅ sub-13 Computation unnecessary (cac

In [12]:
# checkup: print some content which was written to file
import h5py

# set filename
curr_checkup_filename = f"{curr_bids_root}/derivatives/mne-bids-pipeline/sub-01/eeg/sub-01_task-SocialMemoryCuing_report.h5"

# print file name if file exists
!"{ls_command}" "{curr_checkup_filename}" | xargs basename

# import data
curr_checkup_h5file = h5py.File(curr_checkup_filename, 'r')
curr_html_length = len(curr_checkup_h5file["mnepython/key__content/idx_0/key_html"])
htmlContent = ''.join([ chr(curr_checkup_h5file["mnepython/key__content/idx_0/key_html"][idx] ) for idx in range(curr_html_length) ])
curr_checkup_h5file.close()
# print excerpt of html output
print(f"[...]\n{htmlContent[2500:2850]}\n[...]")

# print hint regarding extra data
print("\n===\nmore data: see at..")
# for more data, open the written html report file
!"{ls_command}" ./"{cfg.bids_root.parts[-2]}"/"{cfg.bids_root.parts[-1]}"/derivatives/mne-bids-pipeline/sub-01/eeg/sub-01_task-SocialMemoryCuing_report.html

sub-01_task-SocialMemoryCuing_report.h5
[...]
</details>
        <details open>
            <summary><strong>Data</strong></summary>
            <table class="table table-hover table-striped table-sm table-responsive small">
                
                <tr>
                    <th>Sampling frequency</th>
                    <td>500.00 Hz</td>
                </tr>
                
       
[...]

===
more data: see at..
./data/ds003702/derivatives/mne-bids-pipeline/sub-01/eeg/sub-01_task-SocialMemoryCuing_report.html


In [13]:
curr_steps = "preprocessing/_02_head_pos"
!mne_bids_pipeline --config {config_path} --steps {curr_steps}

┌────────┬ Welcome aboard MNE-BIDS-Pipeline! 👋 ────────────────────────────────
│14:57:21│ 📝 Using configuration: ./mne-bids/config/mne-bids-pipeline_tests.py
└────────┴ 
┌────────┬ init/_01_init_derivatives_dir ───────────────────────────────────────
│14:57:21│ ✅ Output directories already exist …
└────────┴ done (1s)
┌────────┬ init/_02_find_empty_room ────────────────────────────────────────────
│14:57:22│ ⏩ Skipping, empty-room data only relevant for MEG …
└────────┴ done (1s)
┌────────┬ preprocessing/_02_head_pos ──────────────────────────────────────────
│14:57:22│ ⏩ Skipping …
└────────┴ done (1s)


In [14]:
# checkup: not needed, since this step was skipped

In [15]:
curr_steps = "preprocessing/_03_maxfilter"
!mne_bids_pipeline --config {config_path} --steps {curr_steps}

┌────────┬ Welcome aboard MNE-BIDS-Pipeline! 👋 ────────────────────────────────
│14:57:23│ 📝 Using configuration: ./mne-bids/config/mne-bids-pipeline_tests.py
└────────┴ 
┌────────┬ init/_01_init_derivatives_dir ───────────────────────────────────────
│14:57:23│ ✅ Output directories already exist …
└────────┴ done (1s)
┌────────┬ init/_02_find_empty_room ────────────────────────────────────────────
│14:57:24│ ⏩ Skipping, empty-room data only relevant for MEG …
└────────┴ done (1s)
┌────────┬ preprocessing/_03_maxfilter ─────────────────────────────────────────
│14:57:24│ ⏩ Skipping …
└────────┴ done (1s)


In [16]:
# checkup: not needed, since this step was skipped

In [ ]:
curr_steps = "preprocessing/_04_frequency_filter"
!mne_bids_pipeline --config {config_path} --steps {curr_steps}

┌────────┬ Welcome aboard MNE-BIDS-Pipeline! 👋 ────────────────────────────────
│14:57:25│ 📝 Using configuration: ./mne-bids/config/mne-bids-pipeline_tests.py
└────────┴ 
┌────────┬ init/_01_init_derivatives_dir ───────────────────────────────────────
│14:57:25│ ✅ Output directories already exist …
└────────┴ done (1s)
┌────────┬ init/_02_find_empty_room ────────────────────────────────────────────
│14:57:26│ ⏩ Skipping, empty-room data only relevant for MEG …
└────────┴ done (1s)
┌────────┬ preprocessing/_04_frequency_filter ──────────────────────────────────
│14:57:32│ ⏳️ sub-05 Setting EEG channel locations to template montage: standard_1005.
│14:57:32│ ⏳️ sub-05 Dropping channels: EOG, BIP1, BIP2, BIP3, BIP4, BIP5, BIP6, BIP7, BIP8, BIP9, BIP10, BIP11, BIP12, BIP13, BIP14, BIP15, BIP16, BIP17, BIP18, BIP19, BIP20, BIP21, BIP22, BIP23, BIP24
│14:57:35│ ⏳️ sub-04 Setting EEG channel locations to template montage: standard_1005.
│14:57:35│ ⏳️ sub-04 Dropping channels: EOG, BIP1, BIP2,

In [ ]:
import mne
import matplotlib.pyplot as plt

In [ ]:
%matplotlib widget

In [ ]:
# checkup: print spectrum of original and filtered signals, print sample rate of new raw data file
curr_checkup_filename = f"{curr_bids_root}/derivatives/mne-bids-pipeline/sub-01/eeg/sub-01_task-SocialMemoryCuing_proc-filt_raw.fif"
curr_checkup_raw_filtered = mne.io.read_raw_fif(curr_checkup_filename, preload=True)
curr_checkup_raw_filtered.plot_psd()
print("checkup plot showing reduced bandwidth of eeg signals")

In [23]:
# checkup: print file name if the processed file exists
curr_subject_path = f"{curr_bids_root}/derivatives/mne-bids-pipeline/sub-01/eeg"
!"{ls_command}" "{curr_subject_path}"/sub-01_task-SocialMemoryCuing_proc-filt_raw.fif | xargs basename

sub-01_task-SocialMemoryCuing_proc-filt_raw.fif


In [ ]:
curr_steps = "preprocessing/_05_make_epochs"
!mne_bids_pipeline --config {config_path} --steps {curr_steps}

In [ ]:
# checkup: plot epochs
curr_checkup_epochs = mne.read_epochs(f"{curr_subject_path}/sub-01_task-SocialMemoryCuing_epo.fif")
mne.viz.plot_epochs_image(curr_checkup_epochs)

In [ ]:
curr_steps = "preprocessing/_06a_run_ica"
!mne_bids_pipeline --config {config_path} --steps {curr_steps}

In [ ]:
# checkup: plot ica components
curr_checkup_ica = mne.preprocessing.read_ica(f"{curr_subject_path}/sub-01_task-SocialMemoryCuing_ica.fif")
curr_checkup_ica.apply(curr_checkup_raw_filtered)
curr_checkup_ica.plot_components(inst=curr_checkup_raw_filtered)
print("checkup: ICA component plots of the selected subject")

In [ ]:
import mne
import mne_icalabel
from mne.preprocessing import read_ica
import pandas as pd
from mne_bids_pipeline._config_utils import (
    get_subjects,
    get_sessions
)

for subject in get_subjects(cfg):
    for session in get_sessions(cfg):
        paths = ih.get_input_fnames_apply_ica(cfg=cfg, subject=subject, session=session)
        ica = read_ica(paths["ica"])
        raw = mne.io.read_raw_fif(paths["raw"])
        
        label_results = mne_icalabel.label_components(raw, ica, method="iclabel")

        print("\n\nSUBJECT:", subject)
        print(str(ica)) # checkup print of known data about ICA
        print("\nresulting predictions:", label_results["y_pred_proba"]) # checkup print
        print("\nresulting labels:     ", label_results["labels"])       # checkup print
        
        labels = label_results["labels"]
        exclude_idx = [
            idx for idx, label in enumerate(labels) if label not in ["brain", "other"]
        ]
        tsv_data = pd.read_csv(paths["components"], sep="\t")
        
        # checkup: print old content of the file
        print("\nold tsv file content:")
        print(str(tsv_data))
        
        tsv_data.loc[exclude_idx, "status"] = "bad"
        
        # checkup: print updated content of the file
        print("\nnew tsv file content:")
        print(tsv_data)
        
        tsv_data.to_csv(paths["components"], sep="\t", index=False)
    


In [ ]:
# checkup: print list of components given in updated …_proc-ica+components.tsv file
!cat "{curr_subject_path}/sub-01_task-SocialMemoryCuing_proc-ica_components.tsv"
# This file should contain a status like "bad" for eye blinks, or "good" for brain activity, as it is indicated above and visible in the ICA component plots.

In [ ]:
curr_steps = "preprocessing/_07a_apply_ica"
!mne_bids_pipeline --config {config_path} --steps {curr_steps}

In [ ]:
# checkup: plot original (conventionally filtered) epochs
curr_checkup_epochs = mne.read_epochs(f"{curr_subject_path}/sub-01_task-SocialMemoryCuing_epo.fif")
mne.viz.plot_epochs_image(curr_checkup_epochs)
print("checkup: original (top) vs. ica-applied (bottom) epochs")

In [ ]:
# checkup: plot ICA-corrected epochs
curr_checkup_epochs_ica = mne.read_epochs(f"{curr_subject_path}/sub-01_task-SocialMemoryCuing_proc-ica_epo.fif")
mne.viz.plot_epochs_image(curr_checkup_epochs_ica)
print("checkup: original (top) vs. ica-applied (bottom) epochs; Compare the scale in μV.")

In [ ]:
curr_steps = "preprocessing/_08_ptp_reject"
!mne_bids_pipeline --config {config_path} --steps {curr_steps}

In [ ]:
# checkup: plot original (conventionally filtered) epochs
curr_checkup_epochs = mne.read_epochs(f"{curr_subject_path}/sub-01_task-SocialMemoryCuing_epo.fif")
mne.viz.plot_epochs_image(curr_checkup_epochs)
print("checkup: original epochs")

In [ ]:
# checkup: plot cleaned epochs
curr_checkup_epochs_ptp = mne.read_epochs(f"{curr_subject_path}/sub-01_task-SocialMemoryCuing_proc-clean_epo.fif")
mne.viz.plot_epochs_image(curr_checkup_epochs_ptp)
print("checkup: original (top) vs. cleaned epochs (bottom)")

In [ ]:
curr_steps = "sensor"
!mne_bids_pipeline --config {config_path} --steps {curr_steps}

In [ ]:
curr_steps = "source"
!mne_bids_pipeline --config {config_path} --steps {curr_steps}

## Files per Step

- `preprocessing/_01_data_quality`
    ```
    sub-01_task-SocialMemoryCuing_report.h5
    sub-01_task-SocialMemoryCuing_report.h5.lock
    sub-01_task-SocialMemoryCuing_report.html
    ```
- `preprocessing/_04_frequency_filter`
    ```
    sub-01_task-SocialMemoryCuing_proc-filt_raw.fif
    ```
- `preprocessing/_05_make_epochs`
    ```
    sub-01_task-SocialMemoryCuing_epo.fif
    ```
- `preprocessing/_06a_run_ica`

    ```
    sub-01_task-SocialMemoryCuing_ica.fif
    sub-01_task-SocialMemoryCuing_proc-ica_components.tsv
    sub-01_task-SocialMemoryCuing_proc-ica+components_report.html
    ```
- ica labelling: no new file (just changes of `*.tsv` file)
- `preprocessing/_07a_apply_ica`
    ```
    sub-01_task-SocialMemoryCuing_proc-ica_epo.fif
    sub-01_task-SocialMemoryCuing_proc-ica_report.html
    ```
- `preprocessing/_08_ptp_reject`
    ```
    sub-01_task-SocialMemoryCuing_proc-clean_epo.fif
    ```
- `sensor`
    ```
    sub-01_task-SocialMemoryCuing_ave.fif
    sub-01_task-SocialMemoryCuing_proc-avatar+sticks+FullEpochs+rocauc_decoding.mat
    sub-01_task-SocialMemoryCuing_proc-avatar+sticks+FullEpochs+rocauc_decoding.tsv
    sub-01_task-SocialMemoryCuing_proc-avatar+sticks+TimeByTime+rocauc_decoding.mat
    sub-01_task-SocialMemoryCuing_proc-avatar+sticks+TimeByTime+rocauc_decoding.tsv
    
    sub-01_task-SocialMemoryCuing_itc+avatar+tfr.h5
    sub-01_task-SocialMemoryCuing_itc+sticks+tfr.h5
    sub-01_task-SocialMemoryCuing_power+avatar+tfr.h5
    sub-01_task-SocialMemoryCuing_power+sticks+tfr.h5
     
    sub-01_task-SocialMemoryCuing_cov.fif
    sub-01_task-SocialMemoryCuing_proc-avatar+sticks+CSP+rocauc_decoding.xlsx
    ```
- `source`
    ```
    sub-01_task-SocialMemoryCuing_fwd.fif
    sub-01_task-SocialMemoryCuing_trans.fif
    ```
